# Dimensionality reduction techniques comparison
**[Kaciel Béraud](mailto:beraud@lpccaen.in2p3.fr)**, Laboratoire de Physique Corpusculaire de Caen, Caen, France.

Date: **28 September 2024**

# Objectives

This session will aim to learn how to play with some data and do some basic classification using classical methods using the [Scikit Learn](https://scikit-learn.org/stable/) and [UMAP learn](https://umap-learn.readthedocs.io/en/latest/) library.

The objectives of this session is to be able to classify the a dataset into there respective class using different classification technics. You will play with some data to understand the technics and apply what you've learn to a new dataset.

## Required Libraries

We suppose you still have the libraries from last lesson, today you will also needs:

* [Seaborn](https://seaborn.pydata.org/) to plot some heatmaps.

* [UMAP](https://umap-learn.readthedocs.io/en/latest/) to perform Uniform Manifold Approximation and Projection for Dimension Reduction. The package name is `umap-learn`

## Dataset

We will use the Wine dataset from Scikitlearn (UCI). It's composed of 178 wine analysis from 3 different producer. 

Here's the list of the features available: 

* alcohol
* malic_acid
* ash
* alcalinity_of_ash
* magensium
* total_phenols
* flavanoids
* nonflavanoid_phenols
* proanthocyanins
* color_intensity
* hue
* od280od315_of_diluted_wines
* proline

Over 3 different classes, 0, 1 and 2.

One can load and take a look at the dataset using the following code

In [ ]:
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd
from IPython.display import display # to show the dataframes nicely

# You can load the data like
# as_frame = True allow one to get a dataframe
data = load_wine(as_frame = True)

print(data.keys())

frame = data["frame"] # take the frame, which also contains the target class
df = pd.DataFrame(frame) # put it as a pandas

display(df)

print(df.shape)

The loaded dataset is a dictionnary containing the data, features and some metadata. 

The `as_frame` argument allow to keep the column name, which is easier to represent, but for a computation purpose, we will not keep them.

The data is stored using a numpy array, it contains the 13 features with 178 observations and we can convert it to a pandas dataframe to work on them.

You can explore the dataset and try to find some correlation by hands.

## Exercise 1

Explore the data by plotting 1 dimension histogram of the features.

Use the `target` class to select along the group and look at the distribution with and without the selection.

Can you find easy discrimination variable in the data?

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolor

# Explore the data

## Exercise 2

Now try to use a second variable and plot 2 dimensions histogram of the features.

Again, use the `target` class to select along the group and look at the distribution with and without the selection.

Can you find better way of discriminating the data? Is it enough?

In [ ]:
# Explore the data, but in 2D

# Dimension reduction

We can manually discreminate up to 2 or even 3 dimensions, but since we have 13 of them, it's impossible to do by hands. That's why we can use **Dimension Reduction** technics, this consist in finding the distances between the points in a hyperspace. The methods can be viewed as geometric even if we are not in a 3D space

For computation purpose, we do not keep the columns names and only get a numpy matrix. We also do not include the target in the features, which will defeat the purpose of the dimension reduction technics.

Before we start, we can prepare a few visualization functions that will be useful.

In [ ]:
# Override the data to remove the labels name
data = load_wine(as_frame = False)
features = data["data"]
labels = data["target"]
features_names = data["feature_names"]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#%matplotlib widget

def visualize_2d(x, labels):
    sns.scatterplot(x=x[:, 0], y=x[:, 1], hue=labels, s=100, alpha=0.8,
                    palette="viridis", edgecolor="black")
    plt.show()

def visualize_3d(x, labels, elev = 30, azim = 45, roll = 0):
    # Workaround as axis limits are not auto-scaling
    x_norm = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
    x, y, z = x_norm[:, 0], x_norm[:, 1], x_norm[:, 2]

    # Colors
    cmap = plt.get_cmap('viridis', 3)
    color = cmap(labels)
    fig = plt.figure(figsize = (7,7))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x, y, z, marker="o", c = color)
    ax.view_init(elev = elev, azim = azim, roll = roll) # if roll give errors, just remove it from this line
    plt.show()

## Correlation matrix

One of the first thing to understand a dataset is to compute the correlation coefficients matrix and then computing the eigenvectors and eigenvalues. This matrix will show the correlation between each features, it's an easy way to find which feature are corrolated with another one.

This can be done manually or using numpy's functions:

In [ ]:
# We have to transpose the matrix to get the correlation coefficient of the features
cov = np.corrcoef(features.T)
eig_vals, eig_vecs = np.linalg.eig(cov)
plt.imshow(cov, vmin = -1, vmax = 1)
plt.colorbar()
plt.show()
print("Eigenvalues: \n", eig_vals)
print("Eigenvectors: \n", eig_vecs)

Note that we need to transpose the matrix, if you do not make the transpose, you will get the correlation coefficient of each observation with another.

## PCA

The first method of dimension reduction is the PCA meaning "Principal Component Analysis". The data are linearly transform into a new coordinate system to capture the largest variation in the data.

Here using Scikit Learn's decomposition submodule, we can use directly the PCA to look at our features.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
pca_2d = pca.fit_transform(features)
visualize_2d(x = pca_2d, labels = labels)

Which you can also get the eigenvectors and eigenvalues of the PCA. We that the eignevalues correspond to the x and y dispersion.

You also have the two component of the eigenvectors for each feature.

In [ ]:
print("Eigenvalues: \n", pca.explained_variance_ratio_)
print("Eigenvectors: \n", pca.components_)

Setting the number of components of the PCA allow us to find the 2 most important features in the dataset, which are:

In [ ]:
print(f"First most contributing component: '{features_names[np.argmax(pca.components_[0])]}' contributing at: , {pca.explained_variance_ratio_[0]*100:.2f}%")
print(f"Second most contributing component: '{features_names[np.argmax(pca.components_[1])]}' contributing at: {pca.explained_variance_ratio_[1]*100:.2f}%")

We can also increase the number of components to match the number of dimension, it will sort each component into there respective contribution.

In [ ]:
pca = PCA(n_components = 13)
pca_2d = pca.fit_transform(features)
print("Eigenvalues: \n", pca.explained_variance_ratio_)
print("Eigenvectors: \n", pca.components_)

We can see that the amount of information carried by each dimension fall down very quickly.

## 3D Projection

We can represent data up to 3 dimension, using the same snippet of code, but we set the number of component to 3.

With a third component, we might see a separation between the group, which we can visualized with a 3D scatter plot:

In [ ]:
pca = PCA(n_components=3)
pca_3d = pca.fit_transform(features)
visualize_3d(pca_3d, labels)

You can make the plot rotate by adding an angle, the first one is the elevation and the second one is the azymut.

In [ ]:
visualize_3d(pca_3d, labels, 0, 135)

### Classification

After one **Dimension Reduction** analysis, you can add a new dimension in your data which can be a first classification to orientate future steps.

This can be done using **NumPy's array** and a seperation function. In our case, we cannot discreminate easily between class 2 et 3, so we will just separate between class 0 and the other class;

In [ ]:
def separation_function(x, y) :
    if x > 180 :
        r_val = 0
    else :
        r_val = 1
    return r_val

l = []
for pos in pca_2d : # we loop over each pair of points
    l.append(separation_function(pos[0], pos[1]))

np_l = np.array(l)
features = np.c_[features, np_l]


`np.c_[]` allows you to append a column in a **NumPy's matrix** and `np.r_[]` is the row equivalent. Other functions like `np.concatenate` exists using a different syntax, you can choose between them depending on how you handle the data.

After the classification, we can measure the quality of the separation by computing the number of **True positives** versus the number of **False positive**.

In our case we can check the quality of the classification of the `class_0` versus the others:

In [ ]:
print(f"Number of real class 0 observations: {np.count_nonzero(labels == 0)}")
print(f"Number of separated class 0: {np.count_nonzero(features[:,-1] == 0)}") # -1 is the last column
print(f"Number of True Positives: {np.count_nonzero((features[:,-1] == 0) & (labels == 0))}")
print(f"Number of False Positives: {np.count_nonzero((features[:,-1] == 0) & (labels != 0))}")

We see that we only have 2 **False Positives** in the separation of the `class_0` but we still miss 13 observations that are excluded by our cut. When choosing a cut like that, you want to have the highest number of **True Positives** with the lowest amount of **False Positives** and the cut you will take is always a compromised between them.

Doing this comparaison with a multitudes of cut will give you a **ROC curve**.

## MDS

MDS stands for **Multidimensional scaling**, it's another method of Dimension reduction, this time it's a non-linear one.

This method take in input a matrix with the distances between each pair of objects. The result of the MDS will also be affected by the distance algorithm used.

We start by computing the distance in a latent space for each pair of observations:

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances

d_matrix = manhattan_distances(features)
d_matrix.shape

Then, we can plot the matrix nicely, here we only plot 10 pair because the matrix can get too big really quickly.

In [ ]:
import seaborn as sns
print("Distance matrix of the first 10 data points...")
distances = d_matrix[:10, :10]

mask = np.triu(np.ones_like(distances, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(distances, mask=mask, cmap=cmap, vmax=distances.max(), center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Or we can simply plot the whole matrix like we did before;

In [ ]:
plt.imshow(d_matrix)
plt.colorbar()
plt.show()

Note: You can see 3 clusters of point, it's because the data are sorted by targets. If the data was shuffle this structure will not be visible.

Now that we have computed the Manhattan Distances, we can now compute the MDS.

Note it works better with Manhattan distances than the euclidean distance, it's most likely a high dimension problem.

In [ ]:
from sklearn.manifold import MDS

mds = MDS(n_components=3,
          normalized_stress=False,
          metric=True,
          dissimilarity="precomputed",
          random_state=2024,
          eps=1e-9)
mds_3d = mds.fit_transform(d_matrix)
visualize_3d(mds_3d, labels)

We can print the "stress", which is the sum of squared distance of the disparities and the distances for all constrained points. if you put `normalized_stress = True` and `metric = False`, you will have normalized stress in which 0 means "perfect" fit, 0.025 excellent, 0.05 good, 0.1 fair, 0.2 poor.

In [ ]:
print("Stress:", mds.stress_)

Same as before, we can put 2 component instead of 3.

In this case we pass directly the features and the **Euclidean distances** will be used. You can compare the difference of the results using the `dissimilarity` argument.

In [ ]:
mds = MDS(n_components=2, normalized_stress=False, eps=1e-9, random_state=2024, dissimilarity="euclidean")
mds_2d = mds.fit_transform(features)
visualize_2d(mds_2d, labels)

## t-SNE

Another technic is the t-SNE, meaning **t-distributed Stocashtic Neighbor embedding**. It's a statistical method with the same objectives as the ones before but based on another algorithm.

Same as before, we can do this using 2 or 3 components;

In [ ]:
from sklearn.manifold import TSNE

tsne_3d = TSNE(n_components=3, perplexity=10, early_exaggeration=12, learning_rate='auto', init='pca', n_jobs=4).fit_transform(features)
visualize_3d(tsne_3d, labels)

In [ ]:
tsne_2d = TSNE(n_components=2, perplexity=10, early_exaggeration=12, learning_rate='auto', init='random', n_jobs=4).fit_transform(features)
visualize_2d(tsne_2d, labels)

## UMAP

UMAP is one the most recent method, it's a variant of the t-SNE using a [Riemannian](https://en.wikipedia.org/wiki/Riemannian_manifold#Riemannian_metrics_and_Riemannian_manifolds) metric.

In [ ]:
import umap.umap_ as umap

reducer = umap.UMAP(n_components=3, n_neighbors=15, min_dist=0.1, metric='euclidean')
umap_3d = reducer.fit_transform(features)
visualize_3d(umap_3d, labels)

In [ ]:
reducer = reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, metric='euclidean')
umap_2d = reducer.fit_transform(features)
visualize_2d(umap_2d, labels)

In [ ]:
reducer = reducer = umap.UMAP(n_components=3, n_neighbors=15, min_dist=0.1, metric='euclidean')
umap_3d = reducer.fit_transform(features)
visualize_3d(umap_3d, labels)

## Exercise 3

Load the `iris` dataset and use the technics seen before to correctly classify the data.

Add acolumn in the data to add another flag and do it again to get better results.

Quantify your cut by the amount of **True Positives** and **False Positives** counts.

In [ ]:
from sklearn.datasets import load_iris

iris_data = load_iris()